### Day 3: Binary Diagnostic

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day03/puzzle_0_1_sample.ipynb)

[Problem statement](https://adventofcode.com/2021/day/3)

#### Part One. Sample data

In [ ]:
using Microsoft.DotNet.Interactive.Formatting;
Formatter.Register<int>(v => Convert.ToString(v,  2).PadLeft(5, '0'))

In [ ]:
#load "common.fsx"

In [ ]:
#!value --name diagnosticRaw --from-file ./sample

In [ ]:
#!share --from value diagnosticRaw

let reportSample = parseDiagnostic diagnosticRaw

reportSample

index,value
0,00100
1,11110
2,10110
3,10111
4,10101
5,01111
6,00111
7,11100
8,10000
9,11001


Let's try to make some use of binary arithmetic.

For a certain bit we can build the bit mask

In [ ]:
let mask i = 1 <<< i

For the first bit (in terms of the task) it will be

In [ ]:
mask 4

10000

After applying the mask to all the sample values we can count which bit - `0` or `1` prevails for a certain bit.

In [ ]:
let getMostCommon numbers i = 
    let maski = mask i
    let ``0s`` = numbers |> Seq.filter (fun n -> n &&& maski = 0) |> Seq.length
    let ``1s`` = Array.length numbers - ``0s``
    if (``0s`` > ``1s``) then 0 else 1

In [ ]:
[|4..-1..0|] |> Array.map(getMostCommon reportSample) |> Array.map byte // different type to fool the formatter

index,value
0,1
1,0
2,1
3,1
4,0


This way we get prevailing bits for each index and shift them to the corresponding place

In [ ]:
[|4..-1..0|] 
|> Array.map(fun i -> getMostCommon reportSample i <<< i)

index,value
0,10000
1,00000
2,00100
3,00010
4,00000


Actually, shifting `0` is a nonsense, and shifted `1` equals to `maski`.

Then, most common shifted bits can get combined using disjunction

In [ ]:
let getMostCommon numbers i = 
    let maski = mask i
    let ``0s`` = numbers |> Seq.filter (fun n -> n &&& maski = 0) |> Seq.length
    let ``1s`` = Array.length numbers - ``0s``
    if (``0s`` > ``1s``) then 0 else maski

[|4..-1..0|] 
|> Array.map(fun i -> getMostCommon reportSample i)

index,value
0,10000
1,00000
2,00100
3,00010
4,00000


In [ ]:
let γ = 
    [|4..-1..0|] 
    |> Array.map(fun i -> getMostCommon reportSample i)
    |> Array.reduce (|||)

For the epsilon rate we can realize we would get the opposite bits as we did for the gamma rate. So, instead of calculating the reduction of the shifted least common bits we can negate the gamma rate

In [ ]:
let ε = ~~~γ &&& 0b11111
ε

01001

In [ ]:
γ * ε |> display
γ * ε |> uint64 |> display

11000110

198